In [1]:
import json
from pandas.io.json import json_normalize
from pprint import pprint as pprint
import requests
import sys
import urllib
import pandas as pd
import numpy as np
import folium
import os

from folium.plugins import HeatMap
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

API_KEY= "pLqmLsYj1SsyQqgqFZ2Inn3AhOxuTOKVbwOu043-4O5snad4ido3E1XWuG5g_iVsZ6LvtqPusPoV_pBPJ6nILp5vpNH_RDSNVfyHJ3T693QsJNDqCt5oGqT_Jy6kWnYx"

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'


DEFAULT_TERM = 'Restaurants'
DEFAULT_LOCATION = 'Los Angeles, CA'
SEARCH_LIMIT = 50




In [2]:
def request(host, path, api_key, url_params=None):

    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


In [3]:

def search_loc(api_key, term, location, offset):

    url_params = {
        'offset': offset,
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [4]:

def search_coords(api_key, term, latitude, longitude, radius, offset):

    url_params = {
        'offset': offset,
        'term': term.replace(' ', '+'),
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [5]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[str(name[:-1])] = str(x)

    flatten(y)
    return out

In [6]:

def query_api_loc(term, location, offset):
    """ term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search_loc(API_KEY, term, location, offset)
    businesses = response.get('businesses')
    out = pd.DataFrame(businesses)
    return out


In [7]:
def query_api_coords(term, lat, long, radius, offset):
    response = search_coords(API_KEY, term, lat,long,radius, offset)
    #pprint(response)
    businesses = response.get('businesses')
    out = pd.DataFrame(businesses)
    return out

In [35]:
df= query_api_loc("bars", 'placentia', 0)
df2 = query_api_coords("bars", 33.8711, -117.8628, 8000, 0)
#df=df.drop_duplicates(subset = ['id','longitude','latitude'])
#df =df[df.is_closed == False]

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...


In [36]:
df.head()


,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 33.889985, 'longitude': -117.84723}",(714) 961-9396,1045.223407,kellys-korner-tavern-placentia,https://s3-media4.fl.yelpcdn.com/bphoto/OE5EG1...,False,"{'address1': '907 E Yorba Linda Blvd', 'addres...",Kelly's Korner Tavern,+17149619396,$$,4.5,230,"[delivery, pickup]",https://www.yelp.com/biz/kellys-korner-tavern-...
1,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.88991, 'longitude': -117.84746}",(714) 646-9569,1018.122000,rems-lounge-placentia-3,https://s3-media3.fl.yelpcdn.com/bphoto/I7bJth...,False,"{'address1': '909 E Yorba Linda Blvd', 'addres...",Rem's Lounge,+17146469569,$$,4.0,86,[],https://www.yelp.com/biz/rems-lounge-placentia...
2,"[{'alias': 'breweries', 'title': 'Breweries'}]","{'latitude': 33.8624287319832, 'longitude': -1...",(714) 996-6258,3318.147235,the-bruery-placentia-2,https://s3-media4.fl.yelpcdn.com/bphoto/Peenff...,False,"{'address1': '717 Dunn Way', 'address2': '', '...",The Bruery,+17149966258,$$,4.5,872,[],https://www.yelp.com/biz/the-bruery-placentia-...
3,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.8670486, 'longitude': -117.819...",(714) 528-7000,3261.568923,clubhouse-grille-and-sports-bar-placentia-2,https://s3-media2.fl.yelpcdn.com/bphoto/AhKknE...,False,"{'address1': '2053 E Orangethorpe Ave', 'addre...",Clubhouse Grille & Sports Bar,+17145287000,$,4.0,65,[],https://www.yelp.com/biz/clubhouse-grille-and-...
4,"[{'alias': 'sportsbars', 'title': 'Sports Bars'}]","{'latitude': 33.9041529687571, 'longitude': -1...",(714) 528-9888,2704.502411,group-therapy-pub-placentia,https://s3-media2.fl.yelpcdn.com/bphoto/SxfkqM...,False,"{'address1': '1265 E Imperial Hwy', 'address2'...",Group Therapy Pub,+17145289888,$,3.5,36,[],https://www.yelp.com/biz/group-therapy-pub-pla...


In [37]:
df = df.reindex( columns = df.columns.tolist() + ['Latitude','Longitude']) 
df

,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,Latitude,Longitude
0,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 33.889985, 'longitude': -117.84723}",(714) 961-9396,1045.223407,kellys-korner-tavern-placentia,https://s3-media4.fl.yelpcdn.com/bphoto/OE5EG1...,False,"{'address1': '907 E Yorba Linda Blvd', 'addres...",Kelly's Korner Tavern,+17149619396,$$,4.5,230,"[delivery, pickup]",https://www.yelp.com/biz/kellys-korner-tavern-...,NaN,NaN
1,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.88991, 'longitude': -117.84746}",(714) 646-9569,1018.122000,rems-lounge-placentia-3,https://s3-media3.fl.yelpcdn.com/bphoto/I7bJth...,False,"{'address1': '909 E Yorba Linda Blvd', 'addres...",Rem's Lounge,+17146469569,$$,4.0,86,[],https://www.yelp.com/biz/rems-lounge-placentia...,NaN,NaN
2,"[{'alias': 'breweries', 'title': 'Breweries'}]","{'latitude': 33.8624287319832, 'longitude': -1...",(714) 996-6258,3318.147235,the-bruery-placentia-2,https://s3-media4.fl.yelpcdn.com/bphoto/Peenff...,False,"{'address1': '717 Dunn Way', 'address2': '', '...",The Bruery,+17149966258,$$,4.5,872,[],https://www.yelp.com/biz/the-bruery-placentia-...,NaN,NaN
3,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.8670486, 'longitude': -117.819...",(714) 528-7000,3261.568923,clubhouse-grille-and-sports-bar-placentia-2,https://s3-media2.fl.yelpcdn.com/bphoto/AhKknE...,False,"{'address1': '2053 E Orangethorpe Ave', 'addre...",Clubhouse Grille & Sports Bar,+17145287000,$,4.0,65,[],https://www.yelp.com/biz/clubhouse-grille-and-...,NaN,NaN
4,"[{'alias': 'sportsbars', 'title': 'Sports Bars'}]","{'latitude': 33.9041529687571, 'longitude': -1...",(714) 528-9888,2704.502411,group-therapy-pub-placentia,https://s3-media2.fl.yelpcdn.com/bphoto/SxfkqM...,False,"{'address1': '1265 E Imperial Hwy', 'address2'...",Group Therapy Pub,+17145289888,$,3.5,36,[],https://www.yelp.com/biz/group-therapy-pub-pla...,NaN,NaN
5,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...","{'latitude': 33.8688825853356, 'longitude': -1...",(714) 213-8068,8424.225021,the-republic-fullerton,https://s3-media4.fl.yelpcdn.com/bphoto/BRnWaG...,False,"{'address1': '912 Williamson Ave', 'address2':...",The Republic,+17142138068,$,4.5,62,[],https://www.yelp.com/biz/the-republic-fullerto...,NaN,NaN
6,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.891143911493, 'longitude': -11...",(714) 993-1401,2222.214684,brians-original-sports-bar-fullerton-2,https://s3-media2.fl.yelpcdn.com/bphoto/gP2HUv...,False,"{'address1': '1944 N Placentia Ave', 'address2...",Brian's Original Sports Bar,+17149931401,$,4.0,143,[],https://www.yelp.com/biz/brians-original-sport...,NaN,NaN
7,"[{'alias': 'divebars', 'title': 'Dive Bars'}]","{'latitude': 33.8881336, 'longitude': -117.842...",(714) 996-0929,1068.085543,tjs-locker-room-placentia,https://s3-media3.fl.yelpcdn.com/bphoto/aqaCb1...,False,"{'address1': '1164 E Yorba Linda Blvd', 'addre...",TJ's Locker Room,+17149960929,$,3.5,21,[],https://www.yelp.com/biz/tjs-locker-room-place...,NaN,NaN
8,"[{'alias': 'breweries', 'title': 'Breweries'}]","{'latitude': 33.849549, 'longitude': -117.859775}",(714) 660-2537,3607.894893,bottle-logic-brewing-anaheim,https://s3-media4.fl.yelpcdn.com/bphoto/1ipZ5_...,False,"{'address1': '1072 N Armando St', 'address2': ...",Bottle Logic Brewing,+17146602537,$,4.5,575,[],https://www.yelp.com/biz/bottle-logic-brewing-...,NaN,NaN
9,"[{'alias': 'breweries', 'title': 'Breweries'}]","{'latitude': 33.863208770752, 'longitude': -11...",(714) 993-3390,3561.014185,stereo-brewing-placentia,https://s3-media4.fl.yelpcdn.com/bphoto/r8GhXq...,False,"{'address1': '950 S Via Rodeo', 'address2': No...",Stereo Brewing,+17149933390,$,4.5,77,[],https://www.yelp.com/biz/stereo-brewing-placen...,NaN,NaN


In [38]:
df.columns

Index(['categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url', 'Latitude',
       'Longitude'],
      dtype='object')

In [40]:
df['Latitude']=df['coordinates'].apply(lambda x: x['longitude'])
df['Longitude']=df['coordinates'].apply(lambda x: x['latitude'])

In [41]:
df.head()

,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,Latitude,Longitude
0,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 33.889985, 'longitude': -117.84723}",(714) 961-9396,1045.223407,kellys-korner-tavern-placentia,https://s3-media4.fl.yelpcdn.com/bphoto/OE5EG1...,False,"{'address1': '907 E Yorba Linda Blvd', 'addres...",Kelly's Korner Tavern,+17149619396,$$,4.5,230,"[delivery, pickup]",https://www.yelp.com/biz/kellys-korner-tavern-...,-117.847230,33.889985
1,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.88991, 'longitude': -117.84746}",(714) 646-9569,1018.122000,rems-lounge-placentia-3,https://s3-media3.fl.yelpcdn.com/bphoto/I7bJth...,False,"{'address1': '909 E Yorba Linda Blvd', 'addres...",Rem's Lounge,+17146469569,$$,4.0,86,[],https://www.yelp.com/biz/rems-lounge-placentia...,-117.847460,33.889910
2,"[{'alias': 'breweries', 'title': 'Breweries'}]","{'latitude': 33.8624287319832, 'longitude': -1...",(714) 996-6258,3318.147235,the-bruery-placentia-2,https://s3-media4.fl.yelpcdn.com/bphoto/Peenff...,False,"{'address1': '717 Dunn Way', 'address2': '', '...",The Bruery,+17149966258,$$,4.5,872,[],https://www.yelp.com/biz/the-bruery-placentia-...,-117.879275,33.862429
3,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.8670486, 'longitude': -117.819...",(714) 528-7000,3261.568923,clubhouse-grille-and-sports-bar-placentia-2,https://s3-media2.fl.yelpcdn.com/bphoto/AhKknE...,False,"{'address1': '2053 E Orangethorpe Ave', 'addre...",Clubhouse Grille & Sports Bar,+17145287000,$,4.0,65,[],https://www.yelp.com/biz/clubhouse-grille-and-...,-117.819919,33.867049
4,"[{'alias': 'sportsbars', 'title': 'Sports Bars'}]","{'latitude': 33.9041529687571, 'longitude': -1...",(714) 528-9888,2704.502411,group-therapy-pub-placentia,https://s3-media2.fl.yelpcdn.com/bphoto/SxfkqM...,False,"{'address1': '1265 E Imperial Hwy', 'address2'...",Group Therapy Pub,+17145289888,$,3.5,36,[],https://www.yelp.com/biz/group-therapy-pub-pla...,-117.841555,33.904153


In [11]:
coords = pd.DataFrame
lat=df.iloc[0]['coordinates']['latitude']
long=df.iloc[0]['coordinates']['longitude']
for index, row in df.iterrows():
    lat=df.iloc[index]['coordinates']['latitude']
    long=df.iloc[index]['coordinates']['longitude']
   # df[index]['Latitude'] = lat
    #df[index]['Longitude'] = long
    df.set_value(index, 'Latitude', lat)
    df.set_value(index, 'Longitude', long)
    #coords = coords.append(df.iloc[index]['coordinates'], ignore_index=True)
#result = pd.concat([df, Coordinates], axis=1)
df
j = df.iloc[index]['coordinates']
j

{'latitude': 33.83262, 'longitude': -117.9125}

In [18]:
df2

,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,"[{'alias': 'breweries', 'title': 'Breweries'}]","{'latitude': 33.849549, 'longitude': -117.859775}",(714) 660-2537,2412.582652,bottle-logic-brewing-anaheim,https://s3-media4.fl.yelpcdn.com/bphoto/1ipZ5_...,False,"{'address1': '1072 N Armando St', 'address2': ...",Bottle Logic Brewing,+17146602537,$,4.5,575,[],https://www.yelp.com/biz/bottle-logic-brewing-...
1,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.891143911493, 'longitude': -11...",(714) 993-1401,2394.405041,brians-original-sports-bar-fullerton-2,https://s3-media2.fl.yelpcdn.com/bphoto/gP2HUv...,False,"{'address1': '1944 N Placentia Ave', 'address2...",Brian's Original Sports Bar,+17149931401,$,4.0,143,[],https://www.yelp.com/biz/brians-original-sport...
2,"[{'alias': 'breweries', 'title': 'Breweries'}]","{'latitude': 33.8624287319832, 'longitude': -1...",(714) 996-6258,1800.977200,the-bruery-placentia-2,https://s3-media4.fl.yelpcdn.com/bphoto/Peenff...,False,"{'address1': '717 Dunn Way', 'address2': '', '...",The Bruery,+17149966258,$$,4.5,872,[],https://www.yelp.com/biz/the-bruery-placentia-...
3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 33.889985, 'longitude': -117.84723}",(714) 961-9396,2544.711700,kellys-korner-tavern-placentia,https://s3-media4.fl.yelpcdn.com/bphoto/OE5EG1...,False,"{'address1': '907 E Yorba Linda Blvd', 'addres...",Kelly's Korner Tavern,+17149619396,$$,4.5,230,"[delivery, pickup]",https://www.yelp.com/biz/kellys-korner-tavern-...
4,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.86765, 'longitude': -117.88588}",(714) 871-9211,2167.298191,the-swig-bar-fullerton,https://s3-media4.fl.yelpcdn.com/bphoto/sW6f7I...,False,"{'address1': '2466 Fender Ave', 'address2': ''...",The Swig Bar,+17148719211,$,4.0,46,[],https://www.yelp.com/biz/the-swig-bar-fullerto...
5,"[{'alias': 'sportsbars', 'title': 'Sports Bars...","{'latitude': 33.88991, 'longitude': -117.84746}",(714) 646-9569,2521.614577,rems-lounge-placentia-3,https://s3-media3.fl.yelpcdn.com/bphoto/I7bJth...,False,"{'address1': '909 E Yorba Linda Blvd', 'addres...",Rem's Lounge,+17146469569,$$,4.0,86,[],https://www.yelp.com/biz/rems-lounge-placentia...
6,"[{'alias': 'breweries', 'title': 'Breweries'},...","{'latitude': 33.85404, 'longitude': -117.8668}",(714) 225-3206,1932.564267,phantom-ales-anaheim,https://s3-media2.fl.yelpcdn.com/bphoto/Pe2I_O...,False,"{'address1': '1211 N Las Brisas St', 'address2...",Phantom Ales,+17142253206,$,4.0,338,[],https://www.yelp.com/biz/phantom-ales-anaheim?...
7,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...","{'latitude': 33.8313913199616, 'longitude': -1...",,6362.284735,the-blind-rabbit-anaheim,https://s3-media1.fl.yelpcdn.com/bphoto/6ZvRe0...,False,"{'address1': '440 S Anaheim Blvd', 'address2':...",The Blind Rabbit,,$$,4.5,1207,[restaurant_reservation],https://www.yelp.com/biz/the-blind-rabbit-anah...
8,"[{'alias': 'lounges', 'title': 'Lounges'}]","{'latitude': 33.877088, 'longitude': -117.879159}",(714) 579-7400,1650.540248,57-bar-and-grill-fullerton,https://s3-media3.fl.yelpcdn.com/bphoto/dQs1Lm...,False,"{'address1': '2932 E Nutwood Ave', 'address2':...",57 Bar & Grill,+17145797400,$$,5.0,7,[],https://www.yelp.com/biz/57-bar-and-grill-full...
9,"[{'alias': 'bars', 'title': 'Bars'}]","{'latitude': 33.8705810240492, 'longitude': -1...",(657) 217-4100,5653.677906,tribune-fullerton,https://s3-media1.fl.yelpcdn.com/bphoto/qCu8sS...,False,"{'address1': '101 E Commonwealth Ave', 'addres...",Tribune,+16572174100,$$,4.5,97,[],https://www.yelp.com/biz/tribune-fullerton?adj...


#LA SAMPLE
LA_sample = query_api_coords("food", 33.8711, -117.8628, 40000, 0)
offset = 50
count = 1000

while offset <= count:
    df = query_api_coords("bars", 33.8711, -117.8628, 40000, offset)
    #LA_sample = LA_sample.append(df, ignore_index=True)
    LA_sample = pd.concat([LA_sample, df], ignore_index=True)
    offset += 50

LA_sample.to_csv('LA_SAMPLE.csv', sep='\t', encoding='utf-8')
LA_sample

In [25]:
data = (np.random.normal(size=(100, 3)) *
        np.array([[33, 34, 1]]) +
        np.array([[-117, -118, 1]])).tolist()
#LA county bound box
#$$dW1184005$$eW1180919$$fN0342014$$gN0334213
#"Los Angeles","Los Angeles County","POLYGON((-118.951721 32.75004,-118.951721 34.823302,-117.646374 34.823302,-117.646374 32.75004,-118.951721 32.75004))","32.75004,-118.951721","34.823302,-118.951721","34.823302,-117.646374","32.75004,-117.646374","32.75004,-118.951721","34.823302","32.7500","-117.646374","-118.951721" "06","039","00277284","06039"

In [26]:
m = folium.Map([34, -118], tiles='stamentoner', zoom_start=9)

HeatMap(data).add_to(m)

#m.save(os.path.join('results', 'Heatmap.html'))

m


In [27]:
LA = folium.Map(location=[34.0522, -118.2437],
                    zoom_start = 10) 
LA


In [44]:
la_county_edge = os.path.join('data', 'la_county_edge.json')

m = folium.Map(
    location=[33.5700, -118.2437],
    tiles='Stamen Toner',
    zoom_start=8
)


folium.GeoJson(
    la_county_edge,
    name='geojson'
).add_to(m)
#def search_grid(radius, lat, long)
#convert from R

#def search_box(lat1,long1, lat2,lat2) 
#lat1,long1 of sw corner
#lat2,long2 of ne corner


#formula for lat and long degrees to meters?

rad = 12750

lat_a=34.8300
long_a = -118.1275
lat_b = 34.7300
long_b = -117.9300


for x in range(0,6):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a], color ='gold',fill=False).add_to(m)
for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a+.4], color ='purple',fill=False).add_to(m)
for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a-.4], color ='blue',fill=False).add_to(m)
for x in range(0,5):
    if(x==4):
        folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a-.8], color ='blue',fill=False).add_to(m)
    




for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b], color ='black',fill=False).add_to(m)
for x in range(0,6):
    folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b-.4], color ='crimson',fill=False).add_to(m)
for x in range(0,4):
    if(x!=2):
        folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b-.8], color ='green',fill=False).add_to(m)    
m.save('LA county yelp search hex-circle grid.html') 
m

In [29]:
rad = 12750

lat_a=34.8300
long_a = -118.1275
lat_b = 34.7300
long_b = -117.9300


for x in range(0,6):
    temp = lat_a-(.2*x)
    folium.features.Circle(radius=rad,location=[temp, long_a], color ='gold',fill=False).add_to(m)
for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a+.4], color ='purple',fill=False).add_to(m)
for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a-.4], color ='blue',fill=False).add_to(m)
for x in range(0,5):
    if(x==4):
        folium.features.Circle(radius=rad,location=[lat_a-(.2*x), long_a-.8], color ='blue',fill=False).add_to(m)
    




for x in range(0,5):
    folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b], color ='black',fill=False).add_to(m)
for x in range(0,6):
    folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b-.4], color ='crimson',fill=False).add_to(m)
for x in range(0,4):
    if(x!=2):
        folium.features.Circle(radius=rad,location=[lat_b-(.2*x), long_b-.8], color ='green',fill=False).add_to(m)    

In [43]:
us_states = os.path.join('data', 'us-states.json')

geo_json_data = json.load(open(us_states))

FileNotFoundError: [Errno 2] No such file or directory: 'data\\us-states.json'

In [ ]:
import pandas as pd

US_Unemployment_Oct2012 = os.path.join('data', 'US_Unemployment_Oct2012.csv')
unemployment = pd.read_csv(US_Unemployment_Oct2012)

unemployment.head(5)